This notebook is messy.
In last part of this notebook,I tried making mask of any size and any location without making full-size mask of original image.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#https://www.kaggle.com/hirune924/fast-image-region-loading-using-pyvips
!apt-get update
!apt -y install --fix-missing libvips libvips-dev
!pip install pyvips

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import cv2
import matplotlib.pyplot as plt
import gc
import pyvips

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
PATH_TRAIN = '/kaggle/input/hubmap-kidney-segmentation/train/'
PATH_TEST = '/kaggle/input/hubmap-kidney-segmentation/test/'
PATH_INFO='/kaggle/input/hubmap-kidney-segmentation/HuBMAP-20-dataset_information.csv'

df_info = pd.read_csv(PATH_INFO)

def make_related_file_path(idstr,mode='train'):
    path_tiff = None
    path_structure_json = None
    path_json = None
    idstr = idstr[:-5] if idstr.endswith('.tiff') else idstr
    
    if mode == 'train':
        path_tiff = PATH_TRAIN + idstr +'.tiff'
        path_structure_json = PATH_TRAIN + idstr + '-anatomical-structure.json'
        path_json = PATH_TRAIN + idstr + '.json'
    else:
        path_tiff = PATH_TEST + idstr+'.tiff'
        path_structure_json = PATH_TEST + idstr + '-anatomical-structure.json'
    
    img_size = df_info[df_info['image_file'] == idstr+'.tiff']['height_pixels'].values[0] , df_info[df_info['image_file'] == (idstr+'.tiff')]['width_pixels'].values[0]
    
    return path_tiff , path_structure_json , path_json , img_size
        

In [ ]:
#These function may help me.

def read_json(path):
    content = None
    with open(path,'r') as f:
        content = f.read()
        
    if content is None:
        raise(f"Error loading {path} json file.")
    
    return json.loads(content)

In [ ]:
df_train = pd.read_csv('/kaggle/input/hubmap-kidney-segmentation/train.csv')
df_train.head()

In [ ]:
path_tiff,path_structure,path_json,img_size = make_related_file_path(df_train.iloc[0][0])

json_file = read_json(path_json)

In [ ]:
geodata = []
for obj in json_file:
    data = {}
    coords = np.array(obj['geometry']['coordinates'])
    pcentroid = np.mean(coords,axis=1,dtype=np.int)
    pmax = np.max(coords,axis=1)
    pmin = np.min(coords,axis=1)
    
    data['coords'] = coords
    data['pcentroid'] = pcentroid
    data['pmax'] = pmax
    data['pmin'] = pmin
    data['pboxcenter'] = (pmax + pmin)//2
    data['boxlen'] = pmax-pmin
    geodata.append(data)

In [ ]:
data = read_json(path_structure)
polys = []
for index in range(data.__len__()):
    geom = np.array(data[index]['geometry']['coordinates'])
    polys.append(geom)

In [ ]:
shape = img_size
mask_1 = np.zeros(shape)
for i in range(len(polys)):
    cv2.fillPoly(mask_1, polys[i], i+1)

#plt.imshow(mask_1)

# Making minimap

Minimap is created based on anatomical structure json file. I'll not train where minimap says 0. We can use anatomical structure in test file. (Maybe?)

## Method 1 : Using created mask of original size.

In [ ]:
chunk = 1024
h,w = img_size[0], img_size[1]
minimap = np.zeros(shape=(h//chunk,w//chunk))
mh = h // chunk
mw = w // chunk

for _h in range(mh):
    for _w in range(mw):
        ph = _h*chunk
        pw = _w*chunk
        
        minimap[_h,_w] = np.amax(mask_1[ph:ph+chunk,pw:pw+chunk])

In [ ]:
plt.imshow(minimap)

In [ ]:
del mask_1
gc.collect()

## Method 2 : Transform polygon data into resized matrix.

In [ ]:
polys

In [ ]:
resized_polys = [poly // chunk for poly in polys]

minimap2 = np.zeros(shape=(h//chunk,w//chunk))

for i in range(len(resized_polys)):
    cv2.fillPoly(minimap2, resized_polys[i], i+1,4) # Linetype is 4.
    
plt.imshow(minimap2)

# Ploting target mask

In [ ]:
mask_t = np.zeros(shape=shape,dtype=np.byte)
for data in geodata:
    coord = data['coords']
    cv2.fillPoly(mask_t,coord,1)

In [ ]:
plt.imshow(mask_t)

# Let's see some shapes of target

In [ ]:
for data in geodata[:5]:
    coords = data['coords']
    pmin = data['pmin']
    coords = coords - pmin
    mask = np.zeros(shape=(data['boxlen'][0][1],data['boxlen'][0][0]))
    centroid = data['pcentroid'] - pmin
    cv2.fillPoly(mask,coords,1)
    cv2.circle(mask,tuple(centroid[0]),3,(0,255,0),-1)
    plt.imshow(mask)
    plt.show()


# Helper class for manipulate HuBMap Data


In [ ]:
class HuBMapDataPlay:
    def __init__(self,
                 path_base='../input/hubmap-kidney-segmentation/'
                 ,chunksize = 512
                 ,only_in_cortex=False ):
        self.PATH_BASE = path_base
        self.PATH_TRAIN = os.path.join(self.PATH_BASE,'train/')
        self.PATH_TEST = os.path.join(self.PATH_BASE,'test/')
        self.PATH_OUTPUT = './'
        
        #self.ANOTOMICAL_STRUCT_TYPE_MEDULLAR = "Medullar"
        #self.ANOTOMICAL_STRUCT_TYPE_CORTEX = "Cortex"
        #self.ANOTOMICAL_STRUCT_TYPE_STRIPE = "Stripe"
        
        
        self.PATH ={
            'BASE' : self.PATH_BASE,
            'train' : self.PATH_TRAIN,
            'test' : self.PATH_TEST,
            'info' : os.path.join(self.PATH_BASE,'HuBMAP-20-dataset_information.csv')
        }
        
        self.IMG_NAME = {
            'train' : [name for name in os.listdir(self.PATH['train']) if name.endswith('.tiff')],
            'test' : [name for name in os.listdir(self.PATH['test']) if name.endswith('.tiff')]
        }
        
        self.additionaldata = {
            'train' : {},
            'test' : {}
        }
        
        self.chunksize = chunksize
        self.only_in_cortex = only_in_cortex
        self.df_info = pd.read_csv(self.PATH['info'])
        
        for mode in ['train','test'] :
            for img_name in self.IMG_NAME[mode]:
                self.additionaldata[mode][img_name] = self.__get_additional_data(img_name,mode,chunksize is not None or chunksize is not 0)
        
        
    def __get_additional_data(self,idstr,mode='train',create_minimap=True):
        path_tiff = None
        path_structure_json = None
        path_json = None
        
        idstr = idstr[:-5] if idstr.endswith('.tiff') else idstr
        data = {}
    
        if mode == 'train':
            path_tiff = self.PATH_TRAIN + idstr +'.tiff'
            path_structure_json = self.PATH_TRAIN + idstr + '-anatomical-structure.json'
            path_json = self.PATH_TRAIN + idstr + '.json'
        else:
            path_tiff = self.PATH_TEST + idstr+'.tiff'
            path_structure_json = self.PATH_TEST + idstr + '-anatomical-structure.json'
    
        img_size = self.df_info[self.df_info['image_file'] == idstr+'.tiff']['height_pixels'].values[0] , self.df_info[self.df_info['image_file'] == (idstr+'.tiff')]['width_pixels'].values[0]
        
        # get data from *-anatomical-structure.json
        structure_json = self.__read_json(path_structure_json)
        structure_polys = []
        
        for i in range(structure_json.__len__()):
            iscortex = structure_json[i]['properties']['classification']['name']=="Cortex"
            if (structure_json[i]['geometry']['type'] == "Polygon") and (not self.only_in_cortex or iscortex) :
                poly = np.array(structure_json[i]['geometry']['coordinates'][0])
                structure_polys.append(poly)
            elif (structure_json[i]['geometry']['type'] == "MultiPolygon") and (not self.only_in_cortex or iscortex): 
                for poly in structure_json[i]['geometry']['coordinates'][0]:
                    structure_polys.append(np.array(poly,dtype=np.int))
        structure_polys = np.array(structure_polys)
    
    
        # Get data from {idstr}.json
        glomdata = None
        glo_polys = []
        if mode == 'train':
            glomeruli_json = self.__read_json(path_json)
            glomdata = []
            glo_no = 0;
            for obj in glomeruli_json:
                data = {}
                coords = np.array(obj['geometry']['coordinates'][0])
                pcentroid = np.mean(coords,axis=0,dtype=np.int)
                pmax = np.max(coords,axis=0)
                pmin = np.min(coords,axis=0)
                
                data['no']= glo_no
                data['coords'] = coords
                data['pcentroid'] = pcentroid
                data['pmax'] = pmax
                data['pmin'] = pmin
                data['pboxcenter'] = (pmax + pmin)//2
                data['boxlen'] = pmax-pmin
                glomdata.append(data)
                glo_polys.append(coords)
                glo_no = glo_no + 1
        glo_polys = np.array(glo_polys)
        
        
        # Create minimap
        minimap = None
        if create_minimap :
            h,w = img_size[0], img_size[1]
            resized_coords = structure_polys // self.chunksize 
            minimap = np.zeros(shape=((h//self.chunksize)+1,(w//self.chunksize)+1))

            for i in range(len(resized_coords)):
                cv2.fillPoly(minimap, [resized_coords[i]], i+1,4)
                
        # minimap for glu~ polygon
        minimap_glomeruli = None
        if create_minimap and mode=='train' :
            h,w = img_size[0], img_size[1]
            mh , mw = (h // self.chunksize)+1 , (w // self.chunksize) +1
            minimap_glomeruli = [[[] for k in range(mw)] for i in range(mh)]
            for glomeruli in glomdata :
                _pmmax = glomeruli['pmax'] // self.chunksize
                _pmmin = glomeruli['pmin'] // self.chunksize
                for _mw in range(_pmmin[0],_pmmax[0]+1):
                    for _mh in range(_pmmin[1],_pmmax[1]+1):
                        try:
                            minimap_glomeruli[_mh][_mw].append(glomeruli['no'])
                        except IndexError:
                            print(f"""IndexError raised. Glomeruli cannot be located in minimap. Check indexes.
                            Data in {idstr}
                            Glomeruli No : {glomeruli['no']}
                            Minimap size (h,w): {mh},{mw}
                            Position tried to locate (x,y): {_mw} ,{_mh}
                            Pmax , Pmin: {glomeruli['pmax']} , {glomeruli['pmin']}
                            Original Image size (h,w):{h},{w}""")
                            
                       
        data ={
            'id' : idstr,
            'path_tiff' : path_tiff,
            'path_structure_json' : path_structure_json,
            'path_json' : path_json,
            'img_size' : img_size,
            'structure_polys' : structure_polys,
            'glomeruli_data' : glomdata,
            'minimap' : minimap,
            'minimap_glomeruli':minimap_glomeruli,
            'glo_polys':glo_polys
        }
        

        return data
        
    def __read_json(self,path):
        content = None
        with open(path,'r') as f:
            content = f.read()
        
        if content is None:
            raise(f"Error loading {path} json file.")
    
        return json.loads(content)
    
    def draw_mask(self,img_name,pstart,rect): # Only for train data. 
        polygon_numbers = set()
        pstart = np.array(pstart)
        rect = np.array(rect)
        
        pend = pstart+rect
        mps = pstart // self.chunksize
        mpe = pend //self.chunksize
        mask = np.zeros(shape=rect,dtype=np.byte)
        
        data = self.additionaldata['train'][img_name]
        for _mw in range(mps[0],mpe[0]+1):
            for _mh in range(mps[1],mpe[1]+1):
                minimap_glo = data['minimap_glomeruli']
                polygon_numbers.update(minimap_glo[_mh][_mw])
        
        polygons = [data['glo_polys'][poly_no] - pstart for poly_no in polygon_numbers]
        cv2.fillPoly(mask,polygons,1)
        
        return mask
        
    def get_img(self,img_name,pstart,rect):
        path = self.PATH['train']+img_name if img_name in self.IMG_NAME['train'] else self.PATH['train']+img_name
        img = pyvips.Image.new_from_file(path)
        patch = img.crop(pstart[0], pstart[1], rect[0], rect[1])
        np_img = np.ndarray(buffer=patch.write_to_memory(),dtype=np.uint8,shape=[patch.height, patch.width, patch.bands])
        return np_img
    
    def iter_train_chunk(self,img_name,use_minimap=False):
        minimap = self.additionaldata['train'][img_name]['minimap']
        h,w = minimap.shape
        rect = [self.chunksize,self.chunksize]
        for _w in range(w-1):
            for _h in range(h-1):
                
                if use_minimap and (minimap[_h][_w] == 0) :
                    continue
                pstart = [_w*self.chunksize,_h*self.chunksize]
                img = self.get_img(img_name,pstart,rect)
                mask = self.draw_mask(img_name,pstart,rect)
                yield img,mask,_h,_w
                
    def get_glomeruli_img_mask(self,img_name,idx):
        chunkhalf = self.chunksize //2
        glo = self.additionaldata['train'][img_name]['glomeruli_data'][idx]
        pstart = glo['pboxcenter'] - chunkhalf
        img = self.get_img(img_name,pstart,[self.chunksize,self.chunksize])
        msk = self.draw_mask(img_name,pstart,[self.chunksize,self.chunksize])
        return img,msk
      
                
    def save_train_images_as_chunk(self,use_minimap=False):
        for img_name in self.IMG_NAME['train'] :
            name_head = self.PATH_OUTPUT+img_name[:-5]
            print(f"Processing {img_name}")
            for img,msk,h,w in self.iter_train_chunk(img_name,use_minimap):
                np.savez(name_head+f"_{h}_{w}",img=img,msk=msk)
                

In [ ]:
cvtr = HuBMapDataPlay()

In [ ]:
msk1 = cvtr.draw_mask('e79de561c.tiff',[50,600],[10000,10000])
img1 = cvtr.get_img('e79de561c.tiff',[50,600],[10000,10000])

plt.imshow(msk1)
plt.show()
plt.imshow(img1)
plt.show()

In [ ]:
msk2 = cvtr.draw_mask('0486052bb.tiff',[500,2500],[10240,10240])
img2 = cvtr.get_img('0486052bb.tiff',[500,2500],[10240,10240])

plt.imshow(msk2)
plt.show()
plt.imshow(img2)
plt.show()

In [ ]:
%%time
msk3 = cvtr.draw_mask('2f6ecfcdf.tiff',[1500,3000],[20000,20000])

In [ ]:
plt.imshow(msk3)
plt.show()

In [ ]:
%%time
msk4 = cvtr.draw_mask('0486052bb.tiff',[4000,3500],[cvtr.chunksize,cvtr.chunksize])

In [ ]:
%%time
for img,msk,h,w in cvtr.iter_train_chunk('0486052bb.tiff',True):
    if not (h%10) and not(w%10):
        plt.imshow(img)
        plt.show()
        plt.imshow(msk)
        plt.show()

In [ ]:
#cvtr.save_train_images_as_chunk(True)

In [ ]:
for i in range(5):
    img5,msk5=cvtr.get_glomeruli_img_mask('0486052bb.tiff',i*5)
    plt.imshow(img5)
    plt.show()
    plt.imshow(msk5)
    plt.show()